In [6]:
! . ../honor/download_cornell.sh

--2021-06-15 17:16:30--  https://github.com/Conchylicultor/DeepQA/raw/master/data/cornell/movie_conversations.txt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt [following]
--2021-06-15 17:16:31--  https://raw.githubusercontent.com/Conchylicultor/DeepQA/master/data/cornell/movie_conversations.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6760930 (6.4M) [text/plain]
Saving to: 'movie_conversations.txt'

movie_conversations 100%[===================>]   6.45M  5.64MB/s    in 1.1s    

2021-06-15 17:16:32 (5.64 MB/s) - 'movie_c

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import time
import math
from tqdm import tqdm
tf.__version__

'1.4.0'

In [2]:
lines = open('data/cornell/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('data/cornell/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

The sentences that we will be using to train our model.

In [3]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

The sentences' ids, which will be processed to become our input and target data.

In [4]:
conv_lines[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [5]:
id2line = {}
for line in lines:
    line_ = line.split(" +++$+++ ")
    if len(line_) == 5:
        id2line[line_[0]] = line_[4]

In [6]:
convs = []
for line in conv_lines:
    _line = line.split(" +++$+++ ")[-1][1:-1].replace("'", "").replace(" ", "")
    convs.append(_line.split(','))

In [7]:
convs[:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

dialog example

In [8]:
ID = 1488
for i in convs[ID]:
    print(id2line[i])

We've blown the computer!  Elaine!  Set course change!
Set!
Now!
Compute!


Sort the sentences into questions (inputs) and answers (targets)

In [9]:
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

Check if we have loaded the data correctly

In [10]:
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



Compare lengths of questions and answers

In [11]:
print(len(questions))
print(len(answers))

221616
221616


In [12]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [13]:
clean_questions = [clean_text(line) for line in questions]
clean_answers = [clean_text(line) for line in answers]

Take a look at some of the data to ensure that it has been cleaned well.

In [14]:
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [15]:
lengths = [len(line.split()) for line in clean_questions]

In [16]:
lengths = pd.DataFrame(lengths, columns=['counts'])
lengths.describe()

,counts
count,221616.000000
mean,10.684120
std,11.804262
min,0.000000
25%,4.000000
50%,7.000000
75%,13.000000
max,319.000000


In [17]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
23.0
32.0
57.0


Remove questions and answers that are shorter than 2 words and longer than 32 words.

In [18]:
min_line_length = 2
max_line_length = 32

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for question, answers in zip(clean_questions, clean_answers):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length and \
        len(answers.split()) >= min_line_length and len(answers.split()) <= max_line_length:
        short_questions.append(question)
        short_answers.append(answers)

print("clean_answers len = ", len(clean_answers))
print("short_answers len = ", len(short_answers))

clean_answers len =  221616
short_answers len =  168618


Compare the number of lines we will use with the total number of lines.

In [19]:
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 168618
# of answers: 168618
% of data used: 76.09%


Create a dictionary for the frequency of the vocabulary

In [20]:
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
print(len(vocab))

55594


Remove rare words from the vocabulary.

In [21]:
threshold = 7
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 55594
Size of vocab we will use: 13598


In case we want to use a different vocabulary sizes for the source and target text, we can set different threshold values. Nonetheless, we will create dictionaries to provide a unique integer for each word.

In [22]:

questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

Add the unique tokens to the vocabulary dictionaries.

In [23]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

Create dictionaries to map the unique integers to their respective words. i.e. an inverse dictionary for vocab_to_int.

In [24]:
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

Check the length of the dictionaries.

In [25]:
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

13602
13602
13602
13602


Add the end of sentence token to the end of every answer.

In [26]:
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

Convert the text to integers. Replace any words that are not in the respective vocabulary with "UNK"

In [27]:
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        ints.append(questions_vocab_to_int[word] if word in questions_vocab_to_int else questions_vocab_to_int['<UNK>'])
    questions_int.append(ints)

answers_int = []
for question in short_answers:
    ints = []
    for word in question.split():
        ints.append(answers_vocab_to_int[word] if word in answers_vocab_to_int else answers_vocab_to_int['<UNK>'])
    answers_int.append(ints)

Check the lengths

In [28]:
print(len(questions_int))
print(len(answers_int))

168618
168618


Calculate what percentage of all words have been replaced with UNK

In [29]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int['<UNK>']:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 3404792
Number of times <UNK> is used: 89685
Percent of words that are <UNK>: 2.63%


 Sort questions and answers by the length of questions.  
 This will reduce the amount of padding during training  
 Which should speed up training and help to reduce the loss

In [30]:
sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

168618
168618

[717, 3866]
[2904, 4091, 4091, 4091, 11050, 8057, 6775, 2904, 6587, 2552, 13425, 744, 6476, 2314, 13600]

[2904, 5573]
[9478, 4689, 13222, 9585, 8057, 12996, 7777, 13601, 679, 9478, 13600]

[8071, 2965]
[1155, 1546, 10354, 5389, 9478, 11023, 9585, 2239, 13600]



### NN

In [31]:
class AttentionModel(object):
    pass

In [47]:
def model_inputs(self):
    '''Create palceholders for inputs to the model'''
    self.input_data = tf.placeholder(tf.int32, [None, None], name='input')
    self.targets = tf.placeholder(tf.int32, [None, None], name='targets')
    self.lr = tf.placeholder(tf.float32, name='learning_rate')
    # Placeholder for a dropout keep probability.
    self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [51]:
AttentionModel.__model_inputs = classmethod(model_inputs)

In [48]:
def process_encoding_input(self, target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    self.dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

In [52]:
AttentionModel.__process_encoding_input = classmethod(process_encoding_input)

In [54]:
tf.strided_slice?

In [49]:
def encoding_layer(self, rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob=self.keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop]*num_layers)
    _, self.enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw=enc_cell,
                                                   cell_bw=enc_cell,
                                                   sequence_length=sequence_length,
                                                   inputs=rnn_inputs, 
                                                   dtype=tf.float32)

In [53]:
AttentionModel.__encoding_layer = classmethod(encoding_layer)

In [33]:
tf.reset_default_graph()

In [34]:
print(len(questions_vocab_to_int))
print(len(answers_vocab_to_int))

13602
13602


In [35]:
target_vocab_size = len(vocab)
source_vocab_size = len(vocab)
vocab_size = len(answers_int_to_vocab)+1
embed_size = 1024
rnn_size = 1024  
batch_size = 32
num_layers =  3
learning_rate = 0.001
learning_rate_decay = 0.99
min_lr = 0.0001
keep_prob = 0.5
epochs=50
DISPLAY_STEP=30

In [36]:
def lstm(rnn_size, keep_prob, reuse=False):
    lstm =tf.nn.rnn_cell.LSTMCell(rnn_size,reuse=reuse)
    drop =tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return drop

In [37]:
def attention(rnn_size, encoder_outputs, target_sequence_length, dec_cell):
    attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(rnn_size*2,encoder_outputs,
                                                                   memory_sequence_length=target_sequence_length)
    attention_cell = tf.contrib.seq2seq.AttentionWrapper(dec_cell, attention_mechanism,
                                                             attention_layer_size=rnn_size/2)
    return attention_cell

In [38]:
input_data = tf.placeholder(tf.int32, [None, None], name='input')
target_data = tf.placeholder(tf.int32, [None, None], name='target')
input_data_len = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_len')
target_data_len = tf.placeholder(shape=(None, ), dtype=tf.int32, name='target_len')
lr_rate = tf.placeholder(tf.float32, shape=[], name='lr')
keep_prob = tf.placeholder(tf.float32, shape=[], name='keep_prob')

### Class embedding_lookup

In [39]:
encoder_embeddings = tf.Variable(tf.random_uniform([source_vocab_size, embed_size], -1, 1))
encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, input_data)

### Bidirectional_dynamic_rnn

In [40]:
stacked_cells = lstm(rnn_size, keep_prob)

In [41]:
((encoder_fw_outputs,encoder_bw_outputs),
 (encoder_fw_final_state,encoder_bw_final_state)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=stacked_cells, 
                                                                 cell_bw=stacked_cells, 
                                                                 inputs=encoder_embedded, 
                                                                 sequence_length=input_data_len, 
                                                                 dtype=tf.float32)

In [42]:
encoder_outputs = tf.concat((encoder_fw_outputs,encoder_bw_outputs),2)

In [43]:
encoder_outputs

<tf.Tensor 'concat:0' shape=(?, ?, 2048) dtype=float32>

### class LSTMStateTuple

In [44]:
encoder_state_c = tf.concat((encoder_fw_final_state.c,encoder_bw_final_state.c),1)
encoder_state_h = tf.concat((encoder_fw_final_state.h,encoder_bw_final_state.h),1)
encoder_states = tf.nn.rnn_cell.LSTMStateTuple(c=encoder_state_c,h=encoder_state_h)

In [45]:
encoder_states

LSTMStateTuple(c=<tf.Tensor 'concat_1:0' shape=(?, 2048) dtype=float32>, h=<tf.Tensor 'concat_2:0' shape=(?, 2048) dtype=float32>)

### strided_slice

In [46]:
main = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
decoder_input = tf.concat([tf.fill([batch_size, 1],questions_vocab_to_int['<GO>']), main], 1)

In [47]:
decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, embed_size], -1, 1))
decoder_embedded = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)

In [48]:
dec_cell = lstm(rnn_size*2,keep_prob)

In [49]:
dec_cell

### Dense layer

In [50]:
#output layer for decoder
dense_layer = tf.layers.Dense(target_vocab_size)

### Class TrainingHelper

A helper for use during training. Only reads inputs.  
Returned sample_ids are the argmax of the RNN output logits  
dec_cell_inputs=>inputs: A (structure of) input tensors.  
target_data_len=>sequence_length: An int32 vector tensor.


In [51]:
#sampler = tfa.seq2seq.sampler.TrainingSampler()

sampler = tf.contrib.seq2seq.TrainingHelper(decoder_embedded, target_data_len)

### Cell state initializer

#### attention_cell.zero_state

In [52]:
attention_cell = attention(rnn_size, encoder_outputs, target_data_len, dec_cell)
state = attention_cell.zero_state(dtype=tf.float32, batch_size=batch_size)
state = state.clone(cell_state=encoder_states)

### Class BasicDecoder

In [53]:
decoder_train = tf.contrib.seq2seq.BasicDecoder(cell=attention_cell ,helper=sampler,
                                                  initial_state=state,
                                                  output_layer=dense_layer)


### Class dynamic_decode

In [54]:
outputs_train, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder_train,
                                                        impute_finished=True,
                                                        maximum_iterations=tf.reduce_max(target_data_len))

### Greedy Search

A simple approximation is to use a greedy search that selects the most likely word at each step in the output sequence.

This approach has the benefit that it is very fast, but the quality of the final output sequences may be far from optimal.

### Class GreedyEmbeddingHelper

A helper for use during inference.

Uses the argmax of the output (treated as logits) and passes the result through an embedding layer to get the next input.

In [55]:
infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                        tf.fill([batch_size],
                                                                answers_vocab_to_int['<GO>']),
                                                        answers_vocab_to_int['<EOS>'])

In [56]:
decoder_infer = tf.contrib.seq2seq.BasicDecoder(cell=attention_cell ,helper=infer_helper,
                                                  initial_state=state,
                                                  output_layer=dense_layer)

In [57]:
outputs_infer, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder_infer,
                                                        impute_finished=True,
                                                        maximum_iterations=tf.reduce_max(target_data_len))

### Class identity

Return a tensor with the same shape and contents as input.

In [58]:
training_logits = tf.identity(outputs_train.rnn_output, name='logits')
inference_logits = tf.identity(outputs_infer.sample_id, name='predictions')

### Padding and Masking

Now that all samples have a uniform length, the model must be informed that some part of the data is actually padding and should be ignored. That mechanism is masking

### Class sequence_mask

In [59]:
masks = tf.sequence_mask(target_data_len, tf.reduce_max(target_data_len), dtype=tf.float32, name='masks')

### Class sequence_loss

Weighted cross-entropy loss for a sequence of logits.

logits: A Tensor of shape [batch_size, sequence_length, num_decoder_symbols] and dtype float. The logits correspond to the prediction across all classes at each timestep.

targets: A Tensor of shape [batch_size, sequence_length] and dtype int. The target represents the true class at each timestep.

weights: A Tensor of shape [batch_size, sequence_length] and dtype float. weights constitutes the weighting of each prediction in the sequence. When using weights as masking, set all valid timesteps to 1 and all padded timesteps to 0, e.g. a mask returned by tf.sequence_mask.

In [60]:
cost = tf.contrib.seq2seq.sequence_loss(training_logits,target_data,masks)

### Optimizer Adam

In [61]:
optimizer = tf.train.AdamOptimizer(lr_rate)

### Gradient Clipping

Gradient clipping is a technique to prevent exploding gradients in very deep networks, usually in recurrent neural networks. A neural network is a learning algorithm, also called neural network or neural net, that uses a network of functions to understand and translate data input into a specific output. This type of learning algorithm is designed based on the way neurons function in the human brain. There are many ways to compute gradient clipping, but a common one is to rescale gradients so that their norm is at most a particular value. With gradient clipping, pre-determined gradient threshold be introduced, and then gradients norms that exceed this threshold are scaled down to match the norm. This prevents any gradient to have norm greater than the threshold and thus the gradients are clipped. There is an introduced bias in the resulting values from the gradient, but gradient clipping can keep things stable.

### clip_by_value

Clips tensor values to a specified min and max.

Compute gradients of loss for the variables in var_list.

In [62]:
gradients = optimizer.compute_gradients(cost)
capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
train_op = optimizer.apply_gradients(capped_gradients)

### Padding

Before training, we work on the dataset to convert the variable length sequences into fixed length sequences, by padding. We use a few special symbols to fill in the sequence.

In [63]:
def pad_sentence(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

### Accuracy computation

np.pad will take the input array and add the padding based on the shape

np.equalcompare the target and prediction elementwise

np.meanAverage of the inputs given

In [64]:
def get_accuracy(target, logits):
    max_seq = max(len(target[1]), logits.shape[1])
    if max_seq - len(target[1]):
        target = np.pad(
            target,
            [(0,0),(0,max_seq - len(target[1]))],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

### Train and Test data split

As we know input and output will be our questions and answers.Here we are splliting our dataset wrt batch size(128)

In [65]:
train_data = questions_int[batch_size:]
test_data = answers_int[batch_size:]
val_train_data = questions_int[:batch_size]
val_test_data = answers_int[:batch_size]

In [66]:
len(train_data)

168586

### Prepare train test validation set

we need to pad the sentence before use it to validate our model as we seen in padding section. As our vocabulary index already has the word 'PAD' it.checking it us

In [67]:
pad_int = questions_vocab_to_int['<PAD>']

In [68]:
val_batch_x,val_batch_len = pad_sentence(val_train_data,pad_int)
val_batch_y,val_batch_len_y = pad_sentence(val_test_data,pad_int)
val_batch_x = np.array(val_batch_x)
val_batch_y = np.array(val_batch_y)

### Round the length of train data

we need to round the length of train data wrt batch size in order to have equal number of sentence in each batch

In [69]:
no_of_batches = math.floor(len(train_data)//batch_size)
round_no = no_of_batches*batch_size

### Sentence to sequence

So as given below if we have a question sentence 'how are you' it must not be given as it is in our rnn it must be converted into vector

In [70]:
def sentence_to_seq(sentence, vocabs_to_index):
    results = []
    for word in sentence.split(" "):
        if word in vocabs_to_index:
            results.append(vocabs_to_index[word])
        else:
            results.append(vocabs_to_index['<UNK>'])        
    return results

In [71]:
question_sentence = 'where are you'
question_sentence = sentence_to_seq(question_sentence, questions_vocab_to_int)
print(question_sentence)

[7686, 11266, 9478]


## Tf Session Run/Train Model

Only after running tf.global_variables_initializer() in a session our variables hold the values we told them to hold when we declare them (tf.Variable(tf.zeros(...)), tf.Variable(tf.random_normal(...)),...).

#### Session

we need to run the session by providing the optimize which compute gradient wrt cost in each step and all input and target data with its length. It can return prediction of input data and it must be compared with the original target data to get the accuracy of our model.and which will be cumulated further to get total accuracy for all the batches in a single epochs. Also return the loss for each batch which will be cumulated further to get total loss for all the batches in a single epochs

In [72]:
input_data = tf.placeholder(tf.int32, [None, None], name='input')
target_data = tf.placeholder(tf.int32, [None, None], name='target')
# input_data_len = tf.placeholder(tf.int32, [None], name='input_len')
input_data_len = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_len')
# target_data_len = tf.placeholder(tf.int32, [None], name='target_len')
target_data_len = tf.placeholder(shape=(None, ), dtype=tf.int32, name='target_len')
lr_rate = tf.placeholder(tf.float32, shape=[], name='lr')
keep_prob = tf.placeholder(tf.float32, shape=[], name='keep_prob')

In [352]:
save_path = '/root/coursera/natural-language-processing/week5_honor/model_weights/model'
acc_plt = []
loss_plt = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
    #for epoch in range(1):
        #_, summaries_str = sess.run([train_op, summaries_op])
        #fw.add_summary(summaries_str, global_step=i)
        total_accuracy = 0.0
        total_loss = 0.0
        for bs in tqdm(range(0,round_no  ,batch_size)):
        #for bs in tqdm(range(0,32  ,batch_size)):
            index = min(bs+batch_size, round_no )
            #print(bs,index)
      
            #padding done seperately for each batch in training and testing data
            batch_x,len_x = pad_sentence(train_data[bs:index],pad_int)
            batch_y,len_y = pad_sentence(test_data[bs:index],pad_int)
            batch_x = np.array(batch_x)
            batch_y = np.array(batch_y)
            
            pred,loss_f,opt = sess.run([inference_logits,cost,train_op], 
                                      feed_dict={input_data: batch_x,
                                                 target_data: batch_y,
                                                 input_data_len: len_x,
                                                 target_data_len: len_y,
                                                 lr_rate: learning_rate,
                                                 keep_prob: 0.5})
            train_acc = get_accuracy(batch_y, pred)
            total_loss += loss_f 
            total_accuracy+=train_acc
    
        total_accuracy /= (round_no // batch_size)
    
        total_loss /=  (round_no//batch_size)
        acc_plt.append(total_accuracy)
        loss_plt.append(total_loss)
        prediction_logits = sess.run(inference_logits, {input_data: [question_sentence]*batch_size,
                                         input_data_len: [len(question_sentence)]*batch_size,
                                         target_data_len: [len(question_sentence)]*batch_size,              
                                         keep_prob: 0.75,
                                         })[0]
        print('Epoch %d,Average_loss %f, Average Accucracy %f'%(epoch+1,total_loss,total_accuracy))
        print('  Inputs Words: {}'.format([answers_int_to_vocab[i] for i in question_sentence]))
        print('  Replied Words: {}'.format(" ".join([answers_int_to_vocab[i] for i in prediction_logits])))
        print('\n')
        saver = tf.train.Saver() 
        saver.save(sess, save_path)

100%|██████████| 1/1 [00:12<00:00, 12.57s/it]


Epoch 1,Average_loss 0.002074, Average Accucracy 0.000000
  Inputs Words: ['where', 'are', 'you']
  Replied Words: i i i




In [317]:
tf.reset_default_graph()

In [81]:
%tensorboard --logdir save_path

UsageError: Line magic function `%tensorboard` not found.


In [78]:
! pip install tensorflow-tensorboard==0.4

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
     |################################| 1.7 MB 1.7 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-tensorboard
    Found existing installation: tensorflow-tensorboard 1.5.1
    Uninstalling tensorflow-tensorboard-1.5.1:
      Successfully uninstalled tensorflow-tensorboard-1.5.1
